In [1]:
import dask.dataframe as dd
from dask_ml.model_selection import KFold
from dask_ml.cluster import KMeans
from dask_ml.metrics import accuracy_score
from dask.array.slicing import shuffle_slice
import numpy as np
import random

In [2]:
mirai_devices = ['danmini_doorbell', 'ecobee_thermostat', 'philips_B120N10_baby_monitor', 
            'provision_PT_737E_security_camera', 'provision_PT_838_security_camera',
            'simplehome_XCS_1002_WHT_security_camera', 'simplehome_XCS_1003_WHT_security_camera']

benign = ['danmini_doorbell', 'ecobee_thermostat', 'ennio_doorbell', 'philips_B120N10_baby_monitor', 
            'provision_PT_737E_security_camera', 'provision_PT_838_security_camera', 'samsung_SNH_1011_N_webcam',
            'simplehome_XCS_1002_WHT_security_camera', 'simplehome_XCS_1003_WHT_security_camera']

mirai_attacks = ['ack', # automatic scan for vulnerable devices
                 'scan', # ack flood
                 'syn', # syn flood
                 'udp', # udp flood
                 'udpplain'] # optimized udp flood

In [3]:
first = True
mirai_data = None
for device in mirai_devices:
    for i in range(len(mirai_attacks)):
        attack = mirai_attacks[i]
        if first:
            mirai_data = dd.read_csv(f'../data/n_balo_t/{device}/mirai_attacks/{attack}.csv')
            mirai_data['label'] = i
            first = False
        else:
            to_add = dd.read_csv(f'../data/n_balo_t/{device}/mirai_attacks/{attack}.csv')
            to_add['label'] = i
            mirai_data = mirai_data.append(other=to_add)
        

first = True
benign_data = None
for device in benign:
    if first:
        benign_data = dd.read_csv(f'../data/n_balo_t/{device}/benign_traffic.csv')
        first = False
    else:
        to_add = dd.read_csv(f'../data/n_balo_t/{device}/benign_traffic.csv')
        benign_data = benign_data.append(other=to_add)
benign_data['label'] = 100
mirai_data = mirai_data.append(other=benign_data)
a, b, c, d = mirai_data.random_split([0.25, 0.25, 0.25, 0.25], shuffle=True)
data = c.append(d).append(a).append(b)
big_sample, small_sample = data.random_split([0.85, 0.15])

In [6]:
data = small_sample.to_dask_array(lengths=True)
cv = KFold(n_splits=4, shuffle=True)
kfold = {}
i = 0
for train, test in cv.split(data):
    kfold[f'k{i}'] = (train, test)
    i += 1

In [ ]:
fold_predictions = []
clusters = []
for k in kfold.keys():
    current_fold = kfold[k]
    
    train = current_fold[0]
    test = current_fold[1]
    
    train = data[train]
    test = data[test]
    
    X_train = train[ : , :-1]
    y_train = train[ : , -1]
    X_test = test[ : , :-1]
    y_test = test[ : , -1]
    
    model = KMeans(n_clusters=5, init='k-means||', max_iter=300)
    
    model.fit(X_train)
    y_pred = model.predict(X_test)
    
    fold_predictions.append((y_pred, y_test))
    clusters.append(model.cluster_centers_)
    
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)    
    break
    

C:\ProgramData\Anaconda3\envs\471\lib\site-packages\dask\array\slicing.py:1061: PerformanceWarning: Increasing number of chunks by factor of 324
  dtype=x.dtype,
C:\ProgramData\Anaconda3\envs\471\lib\site-packages\dask\array\slicing.py:1061: PerformanceWarning: Increasing number of chunks by factor of 109
  dtype=x.dtype,


In [ ]:
print(fold_predictions)